In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils import data

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing,
                                              EqOddsPostprocessing,
                                              RejectOptionClassification)
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
random_state = 1

mps


pip install 'aif360[AdversarialDebiasing]'


In [2]:
# read in data and split
X = torch.load('inputs/bfw_senet50_face_embeddings.pt').cpu()
y = torch.load('inputs/bfw_senet50_labels.pt').cpu()
df = pd.read_csv('inputs/bfw_senet50_df.csv')
gender = df['reference_gender']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
0,n004721,n004721,asian,asian,male,male,1.0
1,n004721,n004721,asian,asian,male,male,1.0
2,n004721,n004721,asian,asian,male,male,1.0
3,n004721,n004721,asian,asian,male,male,1.0
4,n004721,n004721,asian,asian,male,male,1.0
...,...,...,...,...,...,...,...
38395,n003412,n005685,white,white,male,male,0.0
38396,n008917,n008839,black,black,female,female,0.0
38397,n008452,n008150,asian,asian,male,male,0.0
38398,n004936,n002509,asian,asian,female,female,0.0


In [3]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)

train_X = X[train_split]
X_val = X[val_split]
test_X = X[test_split]

train_y = y[train_split]
y_val = y[val_split]
test_y = y[test_split]

train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]


train_gender = gender[train_split].values
train_gender[train_gender=='male'] = 0
train_gender[train_gender=='female'] = 1
train_gender = train_gender.astype(int)

gender_val = gender[val_split].values
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)

test_gender = gender[test_split].values
test_gender[test_gender=='male'] = 0
test_gender[test_gender=='female'] = 1
test_gender = test_gender.astype(int)

In [4]:
train_matches = train_df[train_df.labels==1]
train_non_matches = train_df[train_df.labels==0]
print('non matches')
print(train_non_matches['reference_gender'].value_counts())
print('matches')
print(train_matches['reference_gender'].value_counts())

female_non_matches = train_non_matches[train_non_matches['reference_gender']=='female']
male_non_matches = train_non_matches[train_non_matches['reference_gender']=='male']

female_matches = train_matches[train_matches['reference_gender']=='female']
male_matches = train_matches[train_matches['reference_gender']=='male']

non matches
female    5804
male      5675
Name: reference_gender, dtype: int64
matches
female    5809
male      5752
Name: reference_gender, dtype: int64


In [5]:
np.random.seed(random_state)
female_matches_sub_idx = female_matches.index[np.random.choice(len(female_matches.index), size=5675, replace=False)]
male_matches_sub_idx = male_matches.index[np.random.choice(len(male_matches.index), size=5675, replace=False)]
female_non_matches_sub_idx = female_non_matches.index[np.random.choice(len(female_non_matches.index), size=5675, replace=False)]
male_non_matches_sub_idx = male_non_matches.index

X_train = torch.cat([X[female_matches_sub_idx],X[male_matches_sub_idx],X[female_non_matches_sub_idx],X[male_non_matches_sub_idx]])
y_train = torch.cat([y[female_matches_sub_idx],y[male_matches_sub_idx],y[female_non_matches_sub_idx],y[male_non_matches_sub_idx]])
gender_train = np.concatenate([gender[female_matches_sub_idx],gender[male_matches_sub_idx],gender[female_non_matches_sub_idx],gender[male_non_matches_sub_idx]])
df_train = pd.concat([df.iloc[female_matches_sub_idx],df.iloc[male_matches_sub_idx],df.iloc[female_non_matches_sub_idx],df.iloc[male_non_matches_sub_idx]])
df_train

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
17999,n000382,n000382,white,white,female,female,1.0
10563,n006634,n006634,asian,asian,female,female,1.0
17735,n001010,n001010,white,white,female,female,1.0
12546,n000460,n000460,black,black,female,female,1.0
12766,n003694,n003694,black,black,female,female,1.0
...,...,...,...,...,...,...,...
24318,n003465,n001976,asian,asian,male,male,0.0
28359,n004960,n009152,black,black,male,male,0.0
26106,n000964,n000512,white,white,male,male,0.0
26500,n009032,n005807,white,white,male,male,0.0


In [6]:
test_split, holdout_split = train_test_split(np.arange(7680),test_size=0.1, random_state=random_state)
X_test = test_X[test_split]
X_holdout = test_X[holdout_split]
y_test = test_y[test_split]
y_holdout = test_y[test_split]
gender_test = test_gender[holdout_split]
gender_holdout = test_gender[holdout_split]
df_test = test_df.iloc[test_split]
df_holdout = test_df.iloc[holdout_split]
df_holdout

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
5873,n008307,n008307,indian,indian,male,male,1.0
24669,n000806,n005599,white,white,female,female,0.0
4525,n002800,n002800,black,black,male,male,1.0
22366,n002273,n000596,black,black,female,female,0.0
19758,n003655,n002638,black,black,female,female,0.0
...,...,...,...,...,...,...,...
4412,n001280,n001280,black,black,male,male,1.0
37015,n000541,n007272,white,white,male,male,0.0
28389,n007046,n001024,indian,indian,female,female,0.0
15345,n003573,n003573,indian,indian,female,female,1.0


In [7]:
df_test

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
30151,n005807,n001817,white,white,male,male,0.0
22257,n000359,n003139,black,black,female,female,0.0
26180,n004525,n001857,indian,indian,male,male,0.0
22691,n006534,n009143,asian,asian,male,male,0.0
35790,n007036,n001209,white,white,male,male,0.0
...,...,...,...,...,...,...,...
17951,n006026,n006026,white,white,female,female,1.0
36983,n007060,n005088,indian,indian,female,female,0.0
2097,n008429,n008429,asian,asian,male,male,1.0
25937,n003258,n000512,white,white,male,male,0.0


In [8]:
val_matches = val_df[val_df.labels==1]
val_non_matches = val_df[val_df.labels==0]
print('non matches')
print(val_non_matches['reference_gender'].value_counts())
print('matches')
print(val_matches['reference_gender'].value_counts())

val_female_non_matches = val_non_matches[val_non_matches['reference_gender']=='female']
val_male_non_matches = val_non_matches[val_non_matches['reference_gender']=='male']

val_female_matches = val_matches[val_matches['reference_gender']=='female']
val_male_matches = val_matches[val_matches['reference_gender']=='male']

non matches
female    1996
male      1883
Name: reference_gender, dtype: int64
matches
male      1925
female    1876
Name: reference_gender, dtype: int64


In [9]:
test_matches = df_test[df_test.labels==1]
test_non_matches = df_test[df_test.labels==0]
print('non matches')
print(test_non_matches['reference_gender'].value_counts())
print('matches')
print(test_matches['reference_gender'].value_counts())

test_female_non_matches = test_non_matches[test_non_matches['reference_gender']=='female']
test_male_non_matches = test_non_matches[test_non_matches['reference_gender']=='male']

test_female_matches = test_matches[test_matches['reference_gender']=='female']
test_male_matches = test_matches[test_matches['reference_gender']=='male']

non matches
male      1742
female    1739
Name: reference_gender, dtype: int64
matches
male      1719
female    1712
Name: reference_gender, dtype: int64


In [10]:
holdout_matches = df_holdout[df_holdout.labels==1]
holdout_non_matches = df_holdout[df_holdout.labels==0]
print('non matches')
print(holdout_non_matches['reference_gender'].value_counts())
print('matches')
print(holdout_matches['reference_gender'].value_counts())

holdout_female_non_matches = holdout_non_matches[holdout_non_matches['reference_gender']=='female']
holdout_male_non_matches = holdout_non_matches[holdout_non_matches['reference_gender']=='male']

holdout_female_matches = holdout_matches[holdout_matches['reference_gender']=='female']
holdout_male_matches = holdout_matches[holdout_matches['reference_gender']=='male']

non matches
male      184
female    177
Name: reference_gender, dtype: int64
matches
male      204
female    203
Name: reference_gender, dtype: int64


In [11]:
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
def AOE(tn_1,fp_1,fn_1,tp_1,tn_0,fp_0,fn_0,tp_0):
    tpr_1 = tp_1/(tp_1+fn_1)
    tpr_0 = tp_0/(tp_0+fn_0)

    fpr_1 = fp_1/(fp_1+tn_1)
    fpr_0 = fp_0/(fp_0+tn_0)


    return (np.abs(fpr_1-fpr_0) + np.abs(tpr_1 - tpr_0))/2

In [ ]:
eer_thresh = 0.487

In [8]:
val_cos = cos_sim(X_val[:,:2048],X_val[:,2048:])
eer_thresh = 0.487

eer_pred = (val_cos>eer_thresh)*1
acc_pred = (val_cos>acc_thresh)*1
df_bias = df_val.drop(['reference_identity','candidate_identity','reference_ethnicity','candidate_ethnicity','candidate_gender'],axis=1)
df_bias['eer_pred'] = eer_pred
df_bias['acc_pred'] = acc_pred
df_bias_female = df_bias[df_bias['reference_gender']=='female']
df_bias_male = df_bias[df_bias['reference_gender']=='male']

female_eer_tn, female_eer_fp, female_eer_fn, female_eer_tp, female_eer_acc = confusion_mat(df_bias_female['eer_pred'], df_bias_female['labels'])
male_eer_tn, male_eer_fp, male_eer_fn, male_eer_tp, male_eer_acc = confusion_mat(df_bias_male['eer_pred'], df_bias_male['labels'])

female_acc_tn, female_acc_fp, female_acc_fn, female_acc_tp, female_acc_acc = confusion_mat(df_bias_female['acc_pred'], df_bias_female['labels'])
male_acc_tn, male_acc_fp, male_acc_fn, male_acc_tp, male_acc_acc = confusion_mat(df_bias_male['acc_pred'], df_bias_male['labels'])

female_eer_tnr = female_eer_tn/(female_eer_tn+female_eer_fp)
female_eer_tpr = female_eer_tp/(female_eer_tp+female_eer_fn)
female_eer_fnr = female_eer_fn/(female_eer_fn+female_eer_tp)
female_eer_fpr = female_eer_fp/(female_eer_tn+female_eer_fp)

male_eer_tnr = male_eer_tn/(male_eer_tn+male_eer_fp)
male_eer_tpr = male_eer_tp/(male_eer_tp+male_eer_fn)
male_eer_fnr = male_eer_fn/(male_eer_fn+male_eer_tp)
male_eer_fpr = male_eer_fp/(male_eer_tn+male_eer_fp)

print(female_eer_fnr/male_eer_fnr)
print(female_eer_fpr/male_eer_fpr)
print(male_eer_tnr/female_eer_tnr)
print(male_eer_tpr/female_eer_tpr)
print((1-female_eer_acc)/(1-male_eer_acc))
print('female fnr:',female_eer_fnr,'male fnr:',male_eer_fnr)
print('female fpr:',female_eer_fpr,'male fpr:',male_eer_fpr)
print('female tnr:',male_eer_tnr,'male tnr:',female_eer_tnr)
print('female tpr:',male_eer_tpr,'male tpr:',female_eer_tpr)
print('female mis class:',(1-female_eer_acc),'male mis class:',(1-male_eer_acc))

1.3033398588123628
1.1766351871283667
1.020805009681539
1.0074374347293613
1.2025738949381306
female fnr: 0.030966364121729845 male fnr: 0.02375923970432946
female fpr: 0.12172088142707241 male fpr: 0.10344827586206896
female tnr: 0.896551724137931 male tnr: 0.8782791185729276
female tpr: 0.9762407602956705 male tpr: 0.9690336358782702
female mis class: 0.07673987827467588 male mis class: 0.06381302521008403


In [9]:
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)
gender_val
val_bias_df = pd.DataFrame(columns=['label','prediction','protected','rounded'])
val_bias_df['label']=y_val.cpu().numpy()
val_bias_df['prediction']=val_cos.cpu().numpy()
val_bias_df['rounded']=eer_pred.cpu().numpy()
val_bias_df['protected']=gender_val
val_bias_df

,label,prediction,protected,rounded
0,1.0,0.751705,0,1
1,0.0,0.202043,0,0
2,0.0,0.411487,0,0
3,1.0,0.832371,0,1
4,1.0,0.813786,0,1
...,...,...,...,...
7582,0.0,0.416221,0,0
7583,0.0,0.180342,0,0
7584,1.0,0.737287,1,1
7585,0.0,0.374224,1,0


In [10]:
unpriveleged_df = val_bias_df[val_bias_df['protected']==1]
unpriveleged_df

,label,prediction,protected,rounded
5,0.0,0.230826,1,0
6,0.0,0.308525,1,0
11,0.0,0.321014,1,0
14,0.0,0.395218,1,0
15,1.0,0.637429,1,1
...,...,...,...,...
7575,1.0,0.674320,1,1
7578,1.0,0.635194,1,1
7579,0.0,0.431155,1,0
7584,1.0,0.737287,1,1


In [11]:
priveleged_df = val_bias_df[val_bias_df['protected']==0]
priveleged_df

,label,prediction,protected,rounded
0,1.0,0.751705,0,1
1,0.0,0.202043,0,0
2,0.0,0.411487,0,0
3,1.0,0.832371,0,1
4,1.0,0.813786,0,1
...,...,...,...,...
7580,1.0,0.791052,0,1
7581,1.0,0.867996,0,1
7582,0.0,0.416221,0,0
7583,0.0,0.180342,0,0


In [12]:
unpriveleged_tn, unpriveleged_fp, unpriveleged_fn, unpriveleged_tp, unpriveleged_acc = confusion_mat(unpriveleged_df['rounded'], unpriveleged_df['label'])
priveleged_tn, priveleged_fp, priveleged_fn, priveleged_tp, priveleged_acc = confusion_mat(priveleged_df['rounded'], priveleged_df['label'])
unpriveleged_fnr = unpriveleged_fn/(unpriveleged_fn+unpriveleged_tp)
unpriveleged_fpr = unpriveleged_fp/(unpriveleged_fp+unpriveleged_tn)
priveleged_fnr = priveleged_fn/(priveleged_fn+priveleged_tp)
priveleged_fpr = priveleged_fp/(priveleged_fp+priveleged_tn)

print('Overall Accuracy:',round(torch.sum(eer_pred == y_val).item()/y_val.size()[0],3))
print('Missclassificaition Rate Ratio:',round((1-unpriveleged_acc)/(1- priveleged_acc),3))
print('FNR Ratio:',round(unpriveleged_fnr/priveleged_fnr,3),'FPR Ratio:',round(unpriveleged_fpr/priveleged_fpr,3))
print('AOE:',round(AOE(unpriveleged_tn,unpriveleged_fp,unpriveleged_fn,unpriveleged_tp,priveleged_tn,priveleged_fp,priveleged_fn,priveleged_tp),3))

Overall Accuracy: 0.93
Missclassificaition Rate Ratio: 1.203
FNR Ratio: 1.303 FPR Ratio: 1.177
AOE: 0.013


In [13]:
def to_dataframe(y_true, y_pred, y_prot):
        y_true, y_pred, y_prot = y_true.float().cpu().numpy(), y_pred.float().cpu().numpy(), y_prot.astype(np.float32)
        df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'y_prot': y_prot})
        dataset = StandardDataset(df, 'y_true', [1.], ['y_prot'], [[1.]])
        dataset.scores = y_pred.reshape(-1, 1)
        return dataset

In [14]:
aif_data = to_dataframe(y_val, eer_pred, gender_val)

In [15]:
eo = EqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                  unprivileged_groups=[{'y_prot': 1.}],seed=random_state)

eo=eo.fit(aif_data,aif_data)

eo_pred = eo.predict(aif_data).labels.reshape(-1)
eo_label = aif_data.labels.reshape(-1)
eo_prot = aif_data.protected_attributes.reshape(-1)

eo_unpriv_ids = (eo_prot==1)
eo_unpriv_pred = eo_pred[eo_unpriv_ids]
eo_unpriv_label = eo_label[eo_unpriv_ids]

eo_priv_ids = (eo_prot==0)
eo_priv_pred = eo_pred[eo_priv_ids]
eo_priv_label = eo_label[eo_priv_ids]

eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp, eo_unpriv_acc = confusion_mat(eo_unpriv_pred, eo_unpriv_label)
eo_unpriv_fpr = eo_unpriv_fp/(eo_unpriv_fp+eo_unpriv_tn)
eo_unpriv_fnr = eo_unpriv_fn/(eo_unpriv_fn+eo_unpriv_tp)

eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp, eo_priv_acc = confusion_mat(eo_priv_pred, eo_priv_label)
eo_priv_fpr = eo_priv_fp/(eo_priv_fp+eo_priv_tn)
eo_priv_fnr = eo_priv_fn/(eo_priv_fn+eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-eo_unpriv_acc)/(1- eo_priv_acc),3))
print('FPR Ratio:',eo_unpriv_fpr/eo_priv_fpr)
print('FNR Ratio:',eo_unpriv_fnr/eo_priv_fnr)
print(np.sum(eo_pred == eo_label)/len(eo_pred))
print('AOE:',round(AOE(eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp,eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp),3))


Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
1.0
AOE: 0.0


In [16]:
cost_constraint = 'weighted'
privileged_groups=[{'y_prot': 0.}],
unprivileged_groups=[{'y_prot': 1.}]
cpp = CalibratedEqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                             unprivileged_groups=[{'y_prot': 1.}],
                                             cost_constraint=cost_constraint,seed=random_state)
cpp=cpp.fit(aif_data,aif_data)
cpp_pred = cpp.predict(aif_data).labels.reshape(-1)
cpp_label = aif_data.labels.reshape(-1)
cpp_prot = aif_data.protected_attributes.reshape(-1)

unpriv_ids = (cpp_prot==1)
unpriv_pred = cpp_pred[unpriv_ids]
unpriv_label = cpp_label[unpriv_ids]

priv_ids = (cpp_prot==0)
priv_pred = cpp_pred[priv_ids]
priv_label = cpp_label[priv_ids]

unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp, unpriv_acc = confusion_mat(unpriv_pred, unpriv_label)
unpriv_fpr = unpriv_fp/(unpriv_fp+unpriv_tn)
unpriv_fnr = unpriv_fn/(unpriv_fn+unpriv_tp)

priv_tn, priv_fp, priv_fn, priv_tp, priv_acc = confusion_mat(priv_pred, priv_label)
priv_fpr = priv_fp/(priv_fp+priv_tn)
priv_fnr = priv_fn/(priv_fn+priv_tp)

print('Missclassificaition Rate Ratio:',round((1-unpriv_acc)/(1- priv_acc),3))
print('FPR Ratio:',round(unpriv_fpr/priv_fpr,3))
print('FNR Ratio:',round(unpriv_fnr/priv_fnr,3))
print('Overall Accuracy:',round(np.sum(cpp_pred == cpp_label)/len(cpp_label),3))
print('AOE:',round(AOE(unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp,priv_tn, priv_fp, priv_fn, priv_tp),3))

Missclassificaition Rate Ratio: 0.977
FPR Ratio: 1.213
FNR Ratio: 0.548
Overall Accuracy: 0.922
AOE: 0.023


In [17]:
ROC = RejectOptionClassification(unprivileged_groups=[{'y_prot': 1.}],
                                         privileged_groups=[{'y_prot': 0.}],
                                         low_class_thresh=0.3, high_class_thresh=0.8,
                                         num_class_thresh=100, num_ROC_margin=100,
                                         metric_name="Average odds difference",
                                         metric_ub=0.01, metric_lb=-0.01)

ROC = ROC.fit(aif_data, aif_data)

ROC_pred = ROC.predict(aif_data).labels.reshape(-1)
ROC_label = aif_data.labels.reshape(-1)
ROC_prot = aif_data.protected_attributes.reshape(-1)

ROC_unpriv_ids = (ROC_prot==1)
ROC_unpriv_pred = ROC_pred[ROC_unpriv_ids]
ROC_unpriv_label = ROC_label[ROC_unpriv_ids]

ROC_priv_ids = (ROC_prot==0)
ROC_priv_pred = ROC_pred[ROC_priv_ids]
ROC_priv_label = ROC_label[ROC_priv_ids]

ROC_unpriv_tn, ROC_unpriv_fp, ROC_unpriv_fn, ROC_unpriv_tp, ROC_unpriv_acc = confusion_mat(ROC_unpriv_pred, ROC_unpriv_label)
ROC_unpriv_fpr = ROC_unpriv_fp/(ROC_unpriv_fp+ROC_unpriv_tn)
ROC_unpriv_fnr = ROC_unpriv_fn/(ROC_unpriv_fn+ROC_unpriv_tp)

ROC_priv_tn, ROC_priv_fp, ROC_priv_fn, ROC_priv_tp, ROC_priv_acc = confusion_mat(ROC_priv_pred, ROC_priv_label)
ROC_priv_fpr = ROC_priv_fp/(ROC_priv_fp+ROC_priv_tn)
ROC_priv_fnr = ROC_priv_fn/(ROC_priv_fn+ROC_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-ROC_unpriv_acc)/(1- ROC_priv_acc),3))
print('FPR Ratio:',round(ROC_unpriv_fpr/ROC_priv_fpr,3))
print('FNR Ratio:',round(ROC_unpriv_fnr/ROC_priv_fnr,3))
print(np.sum(ROC_pred == ROC_label)/len(ROC_pred))
print('AOE:',round(AOE(ROC_unpriv_tn, ROC_unpriv_fp, ROC_unpriv_fn, ROC_unpriv_tp,ROC_priv_tn, ROC_priv_fp, ROC_priv_fn, ROC_priv_tp),3))


Missclassificaition Rate Ratio: 1.203
FPR Ratio: 1.177
FNR Ratio: 1.303
0.92974825359167
AOE: 0.013
